# Neo4J - with Python #
##### Lukas Hinterleitner, Daniel Nepp #####
For a complete documentation visit: https://neo4j.com/developer/python/

---

## Installation ##
### 1. Download and install Neo4j Desktop: ###
https://neo4j.com/download/

Before you can download the installation file you have to provide your data.
If you don't want give your data away, you can type in random stuff :)

An activation key gets displayed while downloading (shown in the picture below).
Keep it, you will need it later during the installation.
<br><br>

![Neo4J Activation Key](screenshots/key.png)
<br><br>

After the installation finished you should see the home screen which is displayed in the image below.
We will get back to it later when we create our first Neo4J graph database.

![Home screen Neo4J Desktop](screenshots/home_screen.png)
<br><br>

### 2. Create a Conda-Environment : ###

`$ conda create --name neo4j python=3.8.2`


### 3. Activate Conda-Environment : ###

`$ conda activate neo4j`


### 4. Install the Neo4j driver : ###
https://anaconda.org/conda-forge/neo4j-python-driver

`$ conda install -c conda-forge neo4j-python-driver`

### 5. Install jupyter notebook : ###

`$ conda install -c anaconda jupyter`

or install jupyter lab

`$ conda install -c conda-forge jupyterlab`

or install jupyter in pycharm...


## Creating Neo4J Database : ##

When creating a Neo4j database you can choose between creating a new local one or connect to a remote DBMS.
In our tutorial we will create a local one.

![create_local_database](screenshots/create_local_database.png)
<br><br>

### Select Version : ###

The Neo4j version can be selected before creating a local database.
The DBMS name and the password will be set at this stage of the installation. 

![create_database](screenshots/create_database.png)
<br><br>

### Database View : ###

The picture below shows the view after the installation process finished.
To open the settings click on the three dots and after that on "manage" in the upper left corner of the database.
By clicking on the start button the Neo4j console will open.

![database_after_installation](screenshots/database_after_installation.png)
<br><br>

### Database Settings : ###

Additional information about the database are listed in the details sections of the settings. In the other sections are more information and further configurations. 

![database_settingsJ Desktop](screenshots/database_settings.png)
<br><br>

## Neo4j Console : ##

The Neo4j console offer some pretty nice stuff. It has a built-in visualization of the console output and much more.

### Neo4j Console Home Screen : ###

The home screen is shown in the picture below.<br>
To get started with the user interface click on "Get started".<br>
To try Neo4j with live data click on "Play guide". <br>
When clicking on "Start querying" you will learn the basics about the Cypher.<br>

![neo4j_browser_home_screen](screenshots/neo4j_browser_home_screen.png)
<br><br>

### Neo4j Guide - View Data : ###

During the guide the browser is always able to display the data. To do this, run the associated command in the input field on the top or just click on the little play buttons listed before each one. 

![guide_after_creation_tables](screenshots/guide_after_creation_tables.png)
<br><br>

### Neo4j Guide - View Data visually as Graphs and Nodes : ###

By far the coolest feature is the visualization of nodes and their corresponding relationships. They can be shown/hided by simple mouse clicks, just play around a little bit. 

![guide_tables_graph](screenshots/guide_tables_graph.png)
<br><br>

---

## Using Neo4j : ##

The code below is a Neo4J Python hello world example! :)

Neo4j uses a NoSQL type query language, therefore the following statements dispatched to the database will look unfamiliar at first. This language is called "CQL" which stand for Cypher-Query-Language, or short : Cypher.
However let's try a "hello neo4j" code snippet first : <br>

In [1]:
from neo4j import GraphDatabase

uri, user, password = 'bolt://localhost:7687', 'neo4j', 'neo4j_'

def _create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                    "SET a.message = $message "
                    "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    greeting = session.write_transaction(_create_and_return_greeting, "Hello World!")
    print(greeting)

driver.close()

Hello World!, from node 1


---
### 1. Create Neo4J Entities : ###

```sql
CREATE (
        TheMatrix:Movie
        {title:'The Matrix', released:1999, tagline:'Welcome to the Real World'}
        )
```

```sql
CREATE (
        Keanu:Person
        {name:'Keanu Reeves', born:1964}
        )
```

---

### 2. Create Neo4J Relations : ###

```sql
(Keanu) - [:ACTED_IN { roles : [ 'Neo' ]}] -> (TheMatrix)
```

```sql
(LillyW) - [:DIRECTED] -> (TheMatrix)
```

---

### 3. Find Neo4J Entities : ###

Find Keanu Reeves - returns single entity:
```sql
MATCH (keanu {name: "Keanu Reeves"}) RETURN keanu
```

Find The Matrix - returns single entity:
```sql
MATCH (theMatrix {title: "The Matrix"}) RETURN theMatrix
```

Find ten people - returns ten entities:
```sql
MATCH (people:Person) RETURN people.name LIMIT 10
```

Find movies produced in the 90's - returns all movies between 1990 and 2000:
```sql
MATCH (nineties:Movie) WHERE nineties.released >= 1990 AND nineties.released < 2000 RETURN nineties.title
```

List all Keanu Reeves movies - returns all movies where Keanu Reeves acted in:
```sql
MATCH (keanu:Person {name: "Keanu Reeves"}) - [:ACTED_IN] -> (keanuMovies) RETURN keanu,keanuMovies
```

Who directed "The Matrix"? - returns all directors of The Matrix:
```sql
MATCH (theMatrix {title: "The Matrix"}) <- [:DIRECTED] - (directors) RETURN directors.name
```

Keanu Reeves' co-actors -- returns all actors who acted in any movie aside Keanu Reeves:
```sql
MATCH (keanu:Person {name:"Keanu Reeves"}) - [:ACTED_IN] -> (m) <- [:ACTED_IN] - (coActors) RETURN coActors.name
```

How people are related to "The Matrix" - returns all people inclusive their relations to the movie The Matrix:
```sql
MATCH (people:Person) - [relatedTo] - (:Movie {title: "The Matrix"}) RETURN people.name, Type(relatedTo), relatedTo
```

Movies and actors up to 4 "hops" away from Kevin Bacon:
```sql
MATCH (bacon:Person {name:"Kevin Bacon"}) - [*1..4] - (hollywood)
RETURN DISTINCT hollywood
```
Note: hollywood here is just a placeholder for any entity

Bacon path, the shortest path of any relationships to Meg Ryan:
```sql
MATCH p=shortestPath(
(bacon:Person {name:"Kevin Bacon"}) - [*] - (meg:Person {name:"Meg Ryan"})
)
RETURN p
```

Extend Tom Hanks co-actors, to find co-co-actors who haven't worked with Tom Hanks:
```sql
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
    (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cocoActors)
WHERE NOT (tom)-[:ACTED_IN]->()<-[:ACTED_IN]-(cocoActors) AND tom <> cocoActors
RETURN cocoActors.name AS Recommended, count(*) AS Strength ORDER BY Strength DESC
```

Find someone to introduce Tom Hanks to Tom Cruise:
```sql
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
  (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cruise:Person {name:"Tom Cruise"})
RETURN tom, m, coActors, m2, cruise
```

---

### 4. Delete Neo4J Graph : ###
Delete all Movie and Person nodes, and their relationships:
```sql
MATCH (n) DETACH DELETE n
```

Note you only need to compare property values like this when first creating relationships
Prove that the Movie Graph is gone
```sql
MATCH (n) RETURN n
```


---
## Let's have some fun : ##

In [2]:
# importing GraphDatabase
from neo4j import GraphDatabase

# Connection data :
uri, user, password = 'bolt://localhost:7687', 'neo4j', 'neo4j_'

# Connecting...
driver = GraphDatabase.driver(uri, auth=(user, password))


---
### Delete complete graph at the beginning of the example: ###

Before paying around with some custom queries containing custom data let's first
clear the database fully.<br>
After the following query the database is completely empty.

In [3]:
# resetting database

with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    _q("MATCH (n) DETACH DELETE n") # remove all graphs and nodes! BE CAREFUL!

    #---------------------------------------
driver.close()

---
### Inserting people: ###

Let's say we want to store people inside our database. <br>
This requirement can be fulfilled with the following code. <br>
We do not have to define a "table" before adding entries. All we have to do is provide a "type label", which in this case is the "Person" defined below... <br>
Neo4j will automatically store "Person" as a new "class/label/table" or simply an "entity".
In fact this is what nodes in a graph database are called : Entites!

Lust like tables, entities in a graph database can also have properties / attributes. <br>
In this case our "Person" entity class will have the peoperty "name" and "born".<br>
Unlike conventional relational databases Neo4j does not enforce what properties different entites have, even if they have the same type label! <br>
If a property exists within a given entity, then it exists, if not, then not.<br>
Yes, it's that simple.

In [4]:
with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    persons = [
        {'name':'Jammie Tullin', 'born':'1999'},
        {'name':'Tina Tuna', 'born':'1995'},
        {'name':'Marry Murry', 'born':'1992'},
        {'name':'Julian Jingle', 'born':'1965'},
        {'name':'Sam Sum', 'born':'1987'},
        {'name':'Lukas Hinterleitner', 'born':'1998'},
        {'name':'Daniel Nepp', 'born':'1997'}
    ]

    for person in persons: # Creating entities : (:= nodes in the graph)
        result = _q("CREATE (p:Person {name:'%s', born:'%s'}) RETURN p" % (person['name'],person['born']))
        for record in result: print("Person created:", record['p']['name'])
 
    #---------------------------------------
driver.close()

Person created: Jammie Tullin
Person created: Tina Tuna
Person created: Marry Murry
Person created: Julian Jingle
Person created: Sam Sum
Person created: Lukas Hinterleitner
Person created: Daniel Nepp


---
### Insert some movies: ###

Now let's do the same for another type of entity, namely : "Movie"! <br>
Later on we are going to create relations between people and movies that describe
the involvement of people in various movies... <br>
The queries below are not too different from the previous ones.

In [5]:
with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    movies = [
        {'title':'Neo4j', 'year':'2020'},
        {'title':'Matrix4j', 'year':'1234'},
        {'title':'Titanic4j', 'year':'2008'},
        {'title':'4j4j', 'year':'2023'},
        {'title':'Python - attack of the snake', 'year':'2004'},
        {'title':'Mr.Bean4j', 'year':'2006'},
    ]

    for movie in movies:
        result = _q("CREATE (m:Movie {title:'%s', year:'%s'}) RETURN m" % (movie['title'],movie['year']))
        for record in result: print("Movie created:", record['m']['title'])

    #---------------------------------------
driver.close()

Movie created: Neo4j
Movie created: Matrix4j
Movie created: Titanic4j
Movie created: 4j4j
Movie created: Python - attack of the snake
Movie created: Mr.Bean4j


---
### Insert relations between people and movies: ###

Relations are very similar to entities.
They have attributes attached to them, as well as a "class" / "label" to which they belong. <br>
(Like "Person" or "Movie" as in our previous queries...)<br>

The big difference is that they connect two entities with oneanother. <br>
Additionally this connection is also directed, meaning that there is a "parent" and a "child" entity.
This has the great benefit that the relation can say something different for each of the two nodes.
A phantastic example of this would be the following : <br>

- `"John" -> "child of" -> "Marry"`

If relations had no direction than the above would not be possible, or let's say nonsensical. <br>
For the entities we created so far, namely those of type : "Person" & "Movie", we'll simple create the relationship "STARRED_IN"! Meaning that a "Person" entity will have the relationship "STARRED_IN" directed to an entity labeld "Movie". <br>

Take a look :


In [6]:
with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    relations = [
        { 'name':'Lukas Hinterleitner', 'type':'STARRED_IN', 'role':'programmer', 'title':'Neo4j' },
        { 'name':'Lukas Hinterleitner', 'type':'DIRECTED', 'role':'producer', 'title':'Neo4j' },
        { 'name':'Lukas Hinterleitner', 'type':'DIRECTED', 'role':'producer', 'title':'Titanic4j' },

        { 'name':'Daniel Nepp', 'type':'STARRED_IN', 'role':'programmer', 'title':'Neo4j' },
        { 'name':'Daniel Nepp', 'type':'DIRECTED', 'role':'producer', 'title':'Neo4j' },
        { 'name':'Daniel Nepp', 'type':'DIRECTED', 'role':'producer', 'title':'Mr.Bean4j' },


        { 'name':'Jammie Tullin', 'type':'STARRED_IN', 'role':'normal actor', 'title':'Python - attack of the snake' },
        { 'name':'Tina Tuna',  'type':'STARRED_IN', 'role':'normal actor', 'title':'Python - attack of the snake' },
        { 'name':'Jammie Tullin', 'type':'STARRED_IN', 'role':'normal actor', 'title':'4j4j' },
        { 'name':'Tina Tuna',  'type':'STARRED_IN', 'role':'normal actor', 'title':'4j4j' },
        { 'name':'Marry Murry', 'type':'STARRED_IN', 'role':'normal actor', 'title':'Mr.Bean4j' },
        { 'name':'Julian Jingle', 'type':'STARRED_IN', 'role':'normal actor', 'title':'Mr.Bean4j' },
        { 'name':'Sam Sum', 'type':'STARRED_IN', 'role':'normal actor', 'title':'Mr.Bean4j' },

        { 'name':'Marry Murry', 'type':'STARRED_IN', 'role':'normal actor', 'title':'Neo4j' },

        { 'name':'Julian Jingle', 'type':'STARRED_IN', 'role':'normal actor', 'title':'Matrix4j' },
        { 'name':'Sam Sum', 'type':'STARRED_IN', 'role':'normal actor', 'title':'Matrix4j' },

        { 'name':'Julian Jingle', 'type':'STARRED_IN', 'role':'normal actor', 'title':'Titanic4j' },
        { 'name':'Sam Sum', 'type':'STARRED_IN', 'role':'normal actor', 'title':'Titanic4j' }
    ]

    for relation in relations:
        result = _q(
            """
                MATCH (p:Person), (m:Movie)
                WHERE p.name = '%s' AND m.title = '%s'
                CREATE (p) - [r:%s { role: '%s' }] -> (m)
                RETURN p.name, type(r), r.role, m.title
            """ % ( relation['name'], relation['title'], relation['type'], relation['role'])
        )
        for record in result: print("Relation created:", record['p.name'], record['type(r)'], "as", record['r.role'], record['m.title'])

    #---------------------------------------
driver.close()

Relation created: Lukas Hinterleitner STARRED_IN as programmer Neo4j
Relation created: Lukas Hinterleitner DIRECTED as producer Neo4j
Relation created: Lukas Hinterleitner DIRECTED as producer Titanic4j
Relation created: Daniel Nepp STARRED_IN as programmer Neo4j
Relation created: Daniel Nepp DIRECTED as producer Neo4j
Relation created: Daniel Nepp DIRECTED as producer Mr.Bean4j
Relation created: Jammie Tullin STARRED_IN as normal actor Python - attack of the snake
Relation created: Tina Tuna STARRED_IN as normal actor Python - attack of the snake
Relation created: Jammie Tullin STARRED_IN as normal actor 4j4j
Relation created: Tina Tuna STARRED_IN as normal actor 4j4j
Relation created: Marry Murry STARRED_IN as normal actor Mr.Bean4j
Relation created: Julian Jingle STARRED_IN as normal actor Mr.Bean4j
Relation created: Sam Sum STARRED_IN as normal actor Mr.Bean4j
Relation created: Marry Murry STARRED_IN as normal actor Neo4j
Relation created: Julian Jingle STARRED_IN as normal actor M

---
### Get co-stars of Daniel: ###

Now that we have created interesting datapoints on our database we'll go over retreiving data. <br>
This is in fact the most exciting part about graph database, namely : Traversing them to get answers to difficultm questions. <br>
One question would be : "What co-stars dies 'Daniel Nepp' have?" <br>
Relational databases are notoriously difficult when such questions need to be answered because they require complex join operations that are hard to write, understand, and debug... <br>
However graph-databases abstract complex key/foreign-key relationship trivialities away by providing the following syntax to query "joins", aka relations! :<br>

`(pel:ParentEntityLabel) - [:RELATION_LABEL] -> (cel:ChildEntityLabel)`<br>

Take a look : 

In [7]:
with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    result = _q( # Note : "<>" means "=="
        """
            MATCH
                (p:Person {name:"Daniel Nepp"} ) - [:STARRED_IN] -> (m) <- [:STARRED_IN] - (coStar)
            WHERE coStar.name <> p.name
            RETURN DISTINCT coStar.name
        """
    )
    for record in result: print("Daniel's co-star found:", record['coStar.name'] )

    #---------------------------------------
driver.close()

Daniel's co-star found: Marry Murry
Daniel's co-star found: Lukas Hinterleitner


---
### Find all people with their year of birth in the movie Neo4j: ###

This way of traversing the graph to find requested result is very flexible.
When it comes to "WHERE" conditions as one would expect them from relational databses then for the most part Neo4J's Cypher is not too different to already familiar concepts : <br> 

In [8]:
with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    movie = "Neo4j"

    result = _q(
        """
            MATCH
                (p:Person) - [r:STARRED_IN] -> (m:Movie)
            WHERE m.title = \"{0}\"
            RETURN p, type(r), r.role
        """.format(movie)
    )

    for record in result:
        print("Star %s born in %s %s as %s the movie %s" % (record['p']['name'], record['p']['born'],
                                                      record['type(r)'], record['r.role'], movie))

    #---------------------------------------
driver.close()

Star Marry Murry born in 1992 STARRED_IN as normal actor the movie Neo4j
Star Daniel Nepp born in 1997 STARRED_IN as programmer the movie Neo4j
Star Lukas Hinterleitner born in 1998 STARRED_IN as programmer the movie Neo4j
